# Download Daily Grids

This script downloads all daily gridMET data. The output is one file per year per variable, e.g. gridMET ERC 2015.

In [1]:
import ee
import numpy as np
import pandas as pd
import datetime
import time
import os
import geemap

In [2]:
ee.Initialize()

## Utility functions

In [3]:
def generate_annual_named_image(collection, band_name, start, end, band_alias=None):
    """Take an image collection and output a single image with one variable and one band for each day.
    """
    band_alias = band_name if not band_alias else band_alias
    collection_year = collection.filterDate(str(start), str(end)).select(band_name)
    band_names = collection_year.aggregate_array("system:time_start").map(lambda x : ee.Date(x).format("YMMdd").cat(f"_{band_alias}"))
    named_image = collection_year.toBands().rename(band_names)
    
    return named_image

## Load data

In [4]:
westside = ee.FeatureCollection("users/aazuspan/westside_ecoregions_dissolve_zuspan")
nwfp = ee.FeatureCollection("users/aazuspan/nwfpbnd")

aoi = nwfp.geometry()

In [5]:
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")

## Constants

In [6]:
CRS = "EPSG:26910"

### Exporting daily gridMET data

In [26]:
export_options = {
    "region": aoi,
    "scale": 4000,
    "crs": CRS,
    "maxPixels": 1e13,
}

for year in range(1979, 2022):
    for band in ["rmin", "vs", "erc"]:
        gridmet_year = generate_annual_named_image(gridmet, band, year, year+1)

        task_gridmet = ee.batch.Export.image(image=gridmet_year, description=f"gridmet_{band}_{year}", config=export_options)
        task_gridmet.start()